Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9


Wrote a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, November 27
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

import lab9_lib

In [2]:
MU = 5
LAMBDA = 20
MUTATION_PROB = 0.2

LENGTH_SOLUTION = 1000
NUMBER_GENERATIONS = 1_500

NUM_ISLANDS = 2

In [3]:
def generate_random_individual():
    ind = np.random.choice([0, 1], size=LENGTH_SOLUTION)
    return ind

def mutate(ind, size=1):
    if random.random() < MUTATION_PROB:
        index = np.random.choice(list(range(len(ind))), size=size, replace=False)
        ind[index] = 1 - ind[index]
    return ind

def reproduce(ind1, ind2):
    new_ind = np.ndarray(shape=ind1.shape)
    for i in range(len(ind1)):
        gene_giver = random.choice([ind1, ind2])
        new_ind[i] = gene_giver[i]
    return new_ind

In [4]:
# Function to perform swapping between islands
def swap_individuals_between_islands(islands):
    island_indices = list(range(NUM_ISLANDS))
    random.shuffle(island_indices)  # Shuffle island indices to randomize swapping order
    for i in range(0, NUM_ISLANDS - 1):
        island1_index, island2_index = island_indices[i], island_indices[i + 1]
        individual_index_island1 = random.randint(0, MU - 1)
        individual_index_island2 = random.randint(0, MU - 1)
        # Swap individuals between islands
        islands[island1_index][individual_index_island1], islands[island2_index][individual_index_island2] = (
            islands[island2_index][individual_index_island2],
            islands[island1_index][individual_index_island1],
        )

In [12]:
def ga(fitness, islands, islands_evals, memoization=False):
    if memoization:
        pop_history = {}
        for i in range(len(islands_evals)):
            for j in range(len(islands_evals[i])):
                pop_history[islands[i][j].tobytes()] = islands_evals[i][j]

    for generation in tqdm(range(NUMBER_GENERATIONS)):
        for island_ix, island in enumerate(islands):
            parents = island
            parents_evals = islands_evals[island_ix]
            offsprings = []
            offsprings_evals = []

            while len(offsprings) < LAMBDA:
                if (max(parents_evals)-min(parents_evals)) > 1e-5:
                    probabilities = [(score-min(parents_evals)) / (max(parents_evals)-min(parents_evals)) for score in parents_evals]
                else:
                    probabilities = [1 for score in parents_evals]
                probabilities = np.array(probabilities)/sum(probabilities)
                p1 = random.choices(parents, k=1, weights=probabilities)[0]
                p2 = random.choices(parents, k=1, weights=probabilities)[0]
                new_ind1 = mutate(reproduce(p1, p2))
                if memoization and pop_history.get(new_ind1.tobytes()) != None:
                    continue
                new_ind1_eval = fitness(new_ind1)
                offsprings.append(new_ind1)

                offsprings_evals.append(new_ind1_eval)
                if memoization:
                    pop_history[new_ind1.tobytes()] = new_ind1_eval

            all_people = parents + offsprings
            all_evals = parents_evals + offsprings_evals
            best_people = np.argsort(all_evals)[::-1]

            parents = []
            parents_evals = []
            for i in range(MU):
                parents.append(all_people[best_people[i]])
                parents_evals.append(all_evals[best_people[i]])
            islands[island_ix] = parents
            islands_evals[island_ix] = parents_evals
            
            if np.max(parents_evals) - 1.0 >= 0:
                print(f"Early stopping {generation=}")
                return parents, parents_evals
            
        if (generation + 1) % 10 == 0:
            swap_individuals_between_islands(islands)

    return parents, parents_evals

Without memoization

In [13]:
fitness = lab9_lib.make_problem(2)
islands = [ [generate_random_individual() for _ in range(50) ] for _ in range(NUM_ISLANDS) ]
islands_evals = [[fitness(x) for x in island] for island in islands ]
parents, parents_evals = ga(fitness, islands, islands_evals, memoization=False)
i_best = np.argmax(parents_evals)
print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)

 70%|██████▉   | 1047/1500 [00:42<00:18, 24.62it/s]

Early stopping generation=1047
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1

With memoization

In [15]:
fitness = lab9_lib.make_problem(2)
islands = [ [generate_random_individual() for _ in range(50) ] for _ in range(NUM_ISLANDS) ]
islands_evals = [[fitness(x) for x in island] for island in islands ]
parents, parents_evals = ga(fitness, islands, islands_evals, memoization=True)
i_best = np.argmax(parents_evals)
print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)

 33%|███▎      | 493/1500 [01:21<02:47,  6.03it/s]

Early stopping generation=493
[0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0.